## Libraries

In [ ]:
%pip install quandl
%pip install yfinance
%pip install seaborn
%pip install pandas
%pip install matplotlib

In [ ]:
import quandl
import json
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import yfinance as yf
import seaborn as sb
sb.set()
from pandas.plotting import scatter_matrix

# For interactive charts
#%matplotlib notebook
# For static charts
%matplotlib inline

# How to read macro-economy data

### Useful links

* [Nasdaq alternative data](https://data.nasdaq.com)

* [Federal reserve economy data documentation](https://data.nasdaq.com/data/FRED-federal-reserve-economic-data/documentation)

    

## Parameters configuration

In [ ]:
sdate = "2006-01-01"
edate = "2021-12-31"
skey = ""
with open("settings.json", 'r') as j:
    skey=json.load(j)["quandl_secret_key"]

# Get Fred data

* **PCE** = Personal consumption expenditure (Monthly)
* **M1** = Monetary supply (Weekly)
* **M1V** = Monetary velocity (Weekly)
* **MEHOINUSA672N** = Real Median Household Income in the United States (Annual)
* **GDP** = Gross Domestic Product (Quaterly)
* **CPIAUCSL** = Consumer Price Index for All Urban Consumers: All Items (Monthly)
* **UNRATE** = Civilian Unemployment Rate (Montly)


In [ ]:
quandl.ApiConfig.api_key = skey

cols = ['FRED/MEHOINUSA672N','FRED/CPIAUCSL','FRED/GDP','FRED/UNRATE','FRED/M1','FRED/PCE', 'FRED/M1V']

FRED=quandl.get(cols, start_date=sdate, end_date=edate)
FRED.columns = cols

SPY = yf.download("^GSPC", start=sdate, end=edate)
FRED.to_csv("data.csv")

# Plot charts

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(15,15))
FRED['FRED/GDP'].dropna().plot(ax=axes[0,0], title="GDP USA")
FRED['FRED/UNRATE'].dropna().plot(ax=axes[0,1], title="Civilian Unemployment Rate")
FRED['FRED/PCE'].dropna().plot(ax=axes[1,0],title="Personal consumption expenditure")
FRED['FRED/MEHOINUSA672N'].dropna().plot(ax=axes[1,1],title="Real Median Household Income")
FRED['FRED/CPIAUCSL'].dropna().plot(ax=axes[2,0],title="Consumer Price Index")
SPY['Close'].dropna().plot(ax=axes[2,1],title="S&P500")
FRED['FRED/M1'].dropna().plot(ax=axes[3,0],title="Monetary supply M1")
FRED['FRED/M1V'].dropna().plot(ax=axes[3,1],title="Monetary velocty")

fig.tight_layout()


# Resample

Resampling dataframes with one month of granularity and merge them

In [ ]:
SPYM = SPY['Close']
SPYM=SPYM.resample('MS').mean()
FREDM = FRED.resample('MS').mean()
FREDM.fillna(inplace=True, method="ffill", axis=0)
DATA = pd.concat([FREDM, SPYM], axis=1, join="inner")
DATA.columns = cols + ["S&P500"]
print(DATA.tail(1))

In [ ]:
DATA.corr()["S&P500"]

In [ ]:
pd.plotting.scatter_matrix(DATA[['S&P500', 'FRED/UNRATE', 'FRED/PCE']], figsize=(15,15))

In [ ]:
DATA[['FRED/UNRATE']].plot.kde(figsize=(13,7))
plt.axvline(DATA[['FRED/UNRATE']].iloc[-1]['FRED/UNRATE'], color='r', linestyle='--', label="UNRATE Current value {0:.2f}".format(DATA[['FRED/UNRATE']].iloc[-1]['FRED/UNRATE']))
plt.axvline(DATA[['FRED/UNRATE']].mean()['FRED/UNRATE'], color='g', linestyle='--', label="UNRATE mean value {0:.2f}".format(DATA[['FRED/UNRATE']].mean()['FRED/UNRATE']))
plt.legend()

# Another definition of PCE

The "core" PCE price index is defined as personal consumption expenditures (PCE) prices excluding food and energy prices. The core PCE price index measures the prices paid by consumers for goods and services without the volatility caused by movements in food and energy prices to reveal underlying inflation trends


In [ ]:
pce_smallest_gradient=DATA[['FRED/PCE']].diff().nsmallest(3, 'FRED/PCE', keep='first')
pce_largest_gradient=DATA[['FRED/PCE']].diff().nlargest(3, 'FRED/PCE', keep='first')
unrate_smallest_gradient=DATA[['FRED/UNRATE']].nsmallest(3, 'FRED/UNRATE', keep='first')
unrate_largest_gradient=DATA[['FRED/UNRATE']].nlargest(3, 'FRED/UNRATE', keep='first')

SPY[['Close']].plot(figsize=(20,10))
plt.axvline(pce_smallest_gradient.index[0], color='r', linestyle='--', label="Worst drop PCE")
plt.axvline(pce_largest_gradient.index[0], color='g', linestyle='--', label="Raise PCE")
plt.axvline(unrate_smallest_gradient.index[0], color='black', linestyle='dotted', label="Worst drop occupation")
plt.axvline(unrate_largest_gradient.index[0], color='orange', linestyle='dotted', label="Raise occupation")
plt.legend()

print('\nTop 3 positive PCE Gradiend')
print(pce_largest_gradient)
print('\n\nTop 3 negative PCE Gradiend')
print(pce_smallest_gradient)
print('\n\nTop 3 best UNRATE')
print(unrate_largest_gradient)
print('\n\nTop 3 worst UNRATE')
print(unrate_smallest_gradient)




# Saving data to file

In [ ]:
DATA.to_csv("macro_data{0}.{1}.csv".format(sdate, edate))